In [40]:
import torch
import torch.cuda
import os
import time
from datetime import datetime
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import StepLR
from torch import Tensor, nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import torch.nn.functional as F
from tqdm.notebook import tqdm
from sklearn.metrics import (
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)

os.environ["CUDA_VISIBLE_DEVICES"] = '0'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [25]:
wpa3_dataset_class_list = ['BF', 'Deauth', 'SAE_CV', 'UGD', 'SAE_AF', 'TSC', 'DG', 'KRACK_DG']
WIDS_class_list = ['benign', 'BF', 'Deauth', 'SAE_CV', 'UGD', 'SAE_AF', 'TSC', 'DG', 'KRACK_DG']
use_feat = {
    "frame.len": "Int64",
    "radiotap.length": "Int64",
    "radiotap.dbm_antsignal": "Int64",
    "wlan.duration": "Int64",
    "radiotap.present.tsft": "Int64",
    "radiotap.channel.freq": "Int64",
    "radiotap.channel.flags.cck": "Int64",
    "radiotap.channel.flags.ofdm": "Int64",
    "wlan.fc.type": "Int64",
    "wlan.fc.subtype": "Int64",
    "wlan.fc.ds": str,
    "wlan.fc.frag": "Int64",
    "wlan.fc.retry": "Int64",
    "wlan.fc.pwrmgt": "Int64",
    "wlan.fc.moredata": "Int64",
    "wlan.fc.protected": "Int64",
    "wlan.fixed.status_code": "Int64",
    "wlan.fixed.auth_seq": str,
    "Label": str,
}

root_path = f"/home/sun10/WIDS"

## Model

In [26]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size: int, num_layers: int, num_classes):
        super(BiLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = True
        self.lstm = nn.LSTM(input_size, hidden_size,
                            num_layers, batch_first=True, bidirectional=self.bidirectional)
        # fully connected layer
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        # define hidden state
        h0 = torch.zeros(self.num_layers*2, x.size(
            0), self.hidden_size).to(x.device)
        # define cell state
        c0 = torch.zeros(self.num_layers*2, x.size(
            0), self.hidden_size).to(x.device)
        out, (h_n, h_c) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

### Loss

In [27]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = (self.alpha * (1 - pt) ** self.gamma * ce_loss)

        if self.reduction == 'mean':
            return torch.mean(focal_loss)
        elif self.reduction == 'sum':
            return torch.sum(focal_loss)
        else:
            return focal_loss

In [42]:
def draw_confusion_matrix(true_test, pre_test, classes, save_to: str):
    train_cm = confusion_matrix(true_test, pre_test, labels=[
                                i for i in range(len(classes))])
    plt.figure(figsize=(8, 6))
    sns.heatmap(train_cm, annot=True, cmap='Blues', fmt='d',
                xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(f"{save_to}/testing_confusion_matrix.png") 

## Training

In [29]:
class Train_LSTM:
    def __init__(
        self, model: nn.Module, data_path: str, label_path: str, subdir: str, hyper_parameter: dict, classes: list
    ):
        self.model = model
        self.data_path = data_path
        self.label_path = label_path
        self.BATCH_SIZE = hyper_parameter["batch_size"]
        self.init_lr = hyper_parameter["learning_rate"]
        self.RL_STEP = hyper_parameter["learning_rate_step"]
        self.INPUT_SIZE = hyper_parameter["input_size"]
        self.HIDDEN_SIZE = hyper_parameter["hidden_size"]
        self.gamma = hyper_parameter["gamma"]
        self.EPOCH = hyper_parameter["epoch"]
        self.classes = classes
        self.valL_loss = list()
        self.best_val_acc = 0

        self.subdir = subdir
        if not os.path.exists(f"{self.subdir}/pth"):
            os.makedirs(f"{self.subdir}/pth")

        self.train_loader, self.valid_loader, self.train_num, self.valid_num = (
            self.input_train_data(
                self.data_path, self.label_path, self.BATCH_SIZE)
        )

        self.train()

    def get_dataLoader(self, data, label, batch_size: int):
        data = torch.tensor(data, dtype=torch.float)
        data_set = TensorDataset(data, torch.tensor(label, dtype=torch.float))
        data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=True)
        for data, batchsize in data_loader:
            print(f"size of input: {data.size()}")
            print(f"size of batch: {batchsize.size()}")
            break
        return data_loader

    def data_normalized(self, data):
        # 對欄位一到四進行 Min-Max normalization
        min_max_scaler = MinMaxScaler()

        column_one = min_max_scaler.fit_transform(data[:, :1])
        column_two = min_max_scaler.fit_transform(data[:, 1:2])
        column_three = min_max_scaler.fit_transform(data[:, 2:3])
        column_four = min_max_scaler.fit_transform(data[:, 3:4])

        # 將 Min-Max normalization 後的部分和 one-hot encoding 後的部分重新組合成最終的資料
        normalized_data = np.concatenate(
            (column_one, column_two, column_three, column_four, data[:, 4:]), axis=1)
        return normalized_data


    def input_train_data(self, data_path: str, label_path: str, batch_size: int):
        print("process dataset....")

        data = np.load(data_path)
        label = np.load(label_path)

        # normalization
        normalized_data = self.data_normalized(data)
        # normalized_data = data

        train_data, val_data, train_label, val_label = train_test_split(
            normalized_data, label, test_size=0.1, stratify=label, random_state=42
        )

        # 分訓練/驗證
        print("-" * 25 + "train data" + "-" * 25)
        for i in set(train_label):
            print(f"{self.classes[i]}: {np.sum(train_label == i )}")

        print("-" * 25 + "valid data" + "-" * 25)
        for i in set(val_label):
            print(f"{self.classes[i]}: {np.sum(val_label == i )}")

        train_loader = self.get_dataLoader(train_data, train_label, batch_size)
        val_loader = self.get_dataLoader(val_data, val_label, batch_size)

        return train_loader, val_loader, len(train_data), len(val_data)

    def validation(self, model: nn.DataParallel, epoch: int, loss):
        val_total_correct = 0  # 總正確預測數
        model = model.cuda(device=device)
        model = model.eval()  # 將模型設置為評估模式
        criterion = FocalLoss()  # 定義交叉熵損失函數

        # 跟蹤每個類別的正確預測數和總樣本數的變量
        class_correct = list(0.0 for i in enumerate(self.classes))
        class_total = list(0.0 for i in enumerate(self.classes))

        with torch.no_grad():  # val 防止在驗證過程中計算梯度
            val_acc_add = 0.0
            val_loss = 0.0
            true_labels = []
            predicted_labels = []

            for inputs, labels in tqdm(
                self.valid_loader
            ):  # 遍歷驗證數據集（self.valid_loader）中的每個批次
                inputs = inputs.cuda(device=device)
                labels = labels.cuda(device=device)
                inputs = inputs.unsqueeze(1)
                outputs = model(inputs)

                # 獲得預測結果。然後計算該批次的損失（val_loss）
                val_loss = criterion(outputs, labels.long())
                val_loss += float(loss.item() * inputs.size(0))
                _, predicted = torch.max(outputs.data, 1)

                val_total_correct += (predicted == labels).sum().item()
                c = (predicted == labels).squeeze()

                # batch size
                for i in range(labels.size(0)):
                    label = int(labels[i])
                    class_correct[label] += c[i].item()
                    class_total[label] += 1

                # Collect true and predicted labels for confusion matrix plotting
                true_labels.extend(labels.cpu().numpy())
                predicted_labels.extend(predicted.cpu().numpy())

            # Call draw_confusion_matrix function
            if epoch == 50:
                draw_confusion_matrix(
                    true_labels, predicted_labels, self.classes
                )

            for i, c in enumerate(self.classes):
                try:
                    val_acc_add += class_correct[i] / class_total[i]
                except ZeroDivisionError:
                    val_acc_add += 0
                try:
                    acc = 100 * class_correct[i] / class_total[i]
                except ZeroDivisionError:
                    acc = 0
                print(f"Valid Accuracy of {c}: {acc:.4f}%")

            val_acc = val_acc_add / len(self.classes)
            val_loss = val_loss.item() / self.valid_num
            print(f"Valid Accuracy: {val_acc:.4f} | Valid Loss: {val_loss:.4f}")


        self.valL_loss.append(val_loss)
        if epoch % 5 == 0:
            if isinstance(model, torch.nn.DataParallel):
                model_state_dict = model.module.state_dict()
            else:
                model_state_dict = model.state_dict()
            torch.save(
                model_state_dict,
                "{}/pth/model-{:.2f}-val_acc-{}-epoch.pth".format(
                    self.subdir, val_acc, epoch
                ),
            )

        if val_acc >= self.best_val_acc:
            self.best_val_acc = val_acc
            with open(self.subdir + "/Best_valid_Acc.txt", "w") as file:
                file.write(
                    "Best Epoch :"
                    + str(epoch + 1)
                    + f"\n Best acc: {self.best_val_acc}"
                )
            if isinstance(model, torch.nn.DataParallel):
                model_state_dict = model.module.state_dict()
            else:
                model_state_dict = model.state_dict()
            torch.save(model_state_dict,
                       f"{self.subdir}/pth/model-best_val_acc.pth")

    def train(self):
        train_loss = list()
        model = self.model.cuda(device=device)
        model = nn.DataParallel(model)
        # loss function
        criterion = nn.CrossEntropyLoss()  # FocalLoss()
        # optimizer
        optimizer = torch.optim.Adam(model.parameters(), lr=self.init_lr)
        # scheduler
        scheduler = StepLR(optimizer, step_size=self.RL_STEP, gamma=self.gamma)

        print("start training ...")
        for epoch in range(self.EPOCH):
            model = model.train()
            localtime = time.asctime(time.localtime(time.time()))
            training_loss = 0.0
            training_corrects = 0
            train_acc_add = 0.0
            train_class_correct = list(0.0 for i in enumerate(self.classes))
            train_class_total = list(0.0 for i in enumerate(self.classes))

            # 在每个批次的迭代中，更新学习率
            scheduler.step()  # 更新学习率
            print("learning rate:", scheduler.get_last_lr())

            print(
                f"\nEpoch: {epoch + 1}/{self.EPOCH} --- < Starting Time : {localtime} >")
            print(
                "-" * len(f"Epoch: {epoch + 1}/{self.EPOCH} --- < Starting Time : {localtime} >"))

            for i, (inputs, labels) in enumerate(tqdm(self.train_loader)):  # train
                optimizer.zero_grad()  # clear gradients for this training step
                inputs = inputs.cuda(device=device)
                labels = labels.cuda(device=device)
                inputs = inputs.unsqueeze(1)  # 增加一個新的維度來表示通道
                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                t = (preds == labels).squeeze()

                for i in range(labels.size(0)):
                    label = int(labels[i])
                    train_class_correct[label] += t[i].item()
                    train_class_total[label] += 1

                loss = criterion(outputs, labels.long())
                loss.backward()  # backpropagation, compute gradients
                optimizer.step()  # apply gradients

                training_loss += float(loss.item() * inputs.size(0))
                training_corrects += torch.sum(preds == labels.data)  # 預測正確數量

            training_loss = training_loss / self.train_num
            training_acc = training_corrects.double() / self.train_num
            train_loss.append(training_loss)

            with open(self.subdir + "/train_Acc_Loss.txt", "a") as file:
                file.write("Epoch :" + str(epoch + 1) +
                           "/" + str(self.EPOCH) + "\n")
                for i, c in enumerate(self.classes):
                    try:
                        train_acc_add += train_class_correct[i] / \
                            train_class_total[i]
                    except ZeroDivisionError:
                        train_acc_add += 0
                    try:
                        acc = 100 * \
                            train_class_correct[i] / train_class_total[i]
                    except ZeroDivisionError:
                        acc = 0

                    file.write(f"Train Accuracy of {c}: {str(acc)}% \n")
                    print("Train Accuracy of %5s : %8.4f %%" % (c, acc))

                train_acc = train_acc_add / len(self.classes)
                file.write(f"Training Accuracy: {str(train_acc)}")
                file.write(f" | Training loss: {str(training_loss)}\n")
                print("Training Accuracy: ", train_acc)

                self.validation(model, epoch, loss)

        print(
            f"Best model save to: ./{self.subdir}/pth/model-best_val_acc.pth")
        parameter_total = sum([param.nelement()
                              for param in model.parameters()])
        print("Number of parameter: %.2fM" % (parameter_total / 1e6))

In [30]:
gamma = 0.1  # 0.1 0.01 0.001
init_lr = 0.001  # 0.01 0.001 1e-3
EPOCH = 50
BATCH_SIZE = 64  # 64 128
RL_STEP = 7
input_size = len(use_feat.keys()) - 1  # 輸入特徵的維度 (扣掉label)
hidden_size = 64
num_layers = 3

hyper_parameter = {
    "batch_size": BATCH_SIZE,
    "epoch": EPOCH,
    "gamma": gamma,
    "learning_rate_step": RL_STEP,
    "learning_rate": init_lr,
    "input_size": input_size,
    "hidden_size": hidden_size,
    "num_layers": num_layers,
}


save_path = f"{root_path}/result"
use_class = WIDS_class_list
train_data_path = f"{root_path}/sampling/train_data.npy"
train_lable_path = f"{root_path}/sampling/train_label.npy"

num_classes = len(use_class)
subdir = f"{save_path}/balance_" + str(
    datetime.strftime(datetime.now(), "%Y%m%d-%H%M%S")
)
model = BiLSTM(
    input_size=hyper_parameter["input_size"],
    hidden_size=hyper_parameter["hidden_size"],
    num_layers=hyper_parameter["num_layers"],
    num_classes=num_classes
)

Train_LSTM(model, train_data_path,
            train_lable_path, subdir, hyper_parameter, use_class)
# 紀錄所有參數
for k, v in hyper_parameter.items():
    with open(f"{subdir}/hyper_parameter.txt", "a") as file:
        file.write(f"{k}={v}\n")


process dataset....
-------------------------train data-------------------------
benign: 9000
BF: 9000
Deauth: 3964
SAE_CV: 3600
UGD: 3600
SAE_AF: 9000
TSC: 3600
DG: 8132
KRACK_DG: 5278
-------------------------valid data-------------------------
benign: 1000
BF: 1000
Deauth: 441
SAE_CV: 400
UGD: 400
SAE_AF: 1000
TSC: 400
DG: 904
KRACK_DG: 586
size of input: torch.Size([64, 18])
size of batch: torch.Size([64])
size of input: torch.Size([64, 18])
size of batch: torch.Size([64])
start training ...
learning rate: [0.001]

Epoch: 1/50 --- < Starting Time : Fri Aug  2 07:53:20 2024 >
------------------------------------------------------------


/home/sun10/anaconda3/envs/IDS/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


  0%|          | 0/863 [00:00<?, ?it/s]

Train Accuracy of benign :  75.9556 %
Train Accuracy of    BF :  97.8000 %
Train Accuracy of Deauth :  81.3572 %
Train Accuracy of SAE_CV :  78.2222 %
Train Accuracy of   UGD :  95.1111 %
Train Accuracy of SAE_AF :  95.6111 %
Train Accuracy of   TSC :  60.1389 %
Train Accuracy of    DG :  89.9287 %
Train Accuracy of KRACK_DG :  80.7313 %
Training Accuracy:  0.8387290203149507


  0%|          | 0/96 [00:00<?, ?it/s]

...

Valid Accuracy of benign: 97.6000%
Valid Accuracy of BF: 100.0000%
Valid Accuracy of Deauth: 100.0000%
Valid Accuracy of SAE_CV: 100.0000%
Valid Accuracy of UGD: 100.0000%
Valid Accuracy of SAE_AF: 99.7000%
Valid Accuracy of TSC: 100.0000%
Valid Accuracy of DG: 96.2389%
Valid Accuracy of KRACK_DG: 97.6109%
Valid Accuracy: 0.9902 | Valid Loss: 0.0040
learning rate: [1.0000000000000006e-10]

Epoch: 50/50 --- < Starting Time : Fri Aug  2 07:57:42 2024 >
-------------------------------------------------------------


  0%|          | 0/863 [00:00<?, ?it/s]

Train Accuracy of benign :  97.7778 %
Train Accuracy of    BF : 100.0000 %
Train Accuracy of Deauth : 100.0000 %
Train Accuracy of SAE_CV : 100.0000 %
Train Accuracy of   UGD : 100.0000 %
Train Accuracy of SAE_AF :  99.5778 %
Train Accuracy of   TSC : 100.0000 %
Train Accuracy of    DG :  96.5937 %
Train Accuracy of KRACK_DG :  97.9727 %
Training Accuracy:  0.9910244181996364


  0%|          | 0/96 [00:00<?, ?it/s]

Valid Accuracy of benign: 97.6000%
Valid Accuracy of BF: 100.0000%
Valid Accuracy of Deauth: 100.0000%
Valid Accuracy of SAE_CV: 100.0000%
Valid Accuracy of UGD: 100.0000%
Valid Accuracy of SAE_AF: 99.7000%
Valid Accuracy of TSC: 100.0000%
Valid Accuracy of DG: 96.2389%
Valid Accuracy of KRACK_DG: 97.6109%
Valid Accuracy: 0.9902 | Valid Loss: 0.0000
Best model save to: .//home/sun10/IDS/upload/result/balance_20240802-075320/pth/model-best_val_acc.pth
Number of parameter: 0.24M


## Testing

In [31]:
def data_normalized(data):
    # 對欄位一到四進行 Min-Max normalization
    min_max_scaler = MinMaxScaler()

    column_one = min_max_scaler.fit_transform(data[:, :1])
    column_two = min_max_scaler.fit_transform(data[:, 1:2])
    column_three = min_max_scaler.fit_transform(data[:, 2:3])
    column_four = min_max_scaler.fit_transform(data[:, 3:4])

    # 將 Min-Max normalization 後的部分和 one-hot encoding 後的部分重新組合成最終的資料
    normalized_data = np.concatenate(
        (column_one, column_two, column_three, column_four, data[:, 4:]), axis=1)
    return normalized_data

# load testing data
data = np.load(f"{root_path}/sampling/test_data.npy")
label = np.load(f"{root_path}/sampling/test_label.npy")
test_num = len(data)

test_data = data_normalized(data)
test_data = torch.tensor(test_data, dtype=torch.float)

test_set = TensorDataset(test_data, torch.tensor(label, dtype=torch.float))
test_loader = DataLoader(test_set, batch_size=hyper_parameter['batch_size'], shuffle=True)


model_path = f"{subdir}/pth/model-best_val_acc.pth"
model = BiLSTM(
    input_size=hyper_parameter["input_size"],
    hidden_size=hyper_parameter["hidden_size"],
    num_layers=hyper_parameter["num_layers"],
    num_classes=len(use_class)
)
model.load_state_dict(torch.load(model_path))
model.cuda(device=device)
model.eval()

true_label = []
pre_label = []
total_correct = 0
total_false = 0
total = 0
class_correct = list(0.0 for i in enumerate(use_class))
class_total = list(0.0 for i in enumerate(use_class))

with torch.no_grad():
    for inputs, labels in tqdm(test_loader):
        inputs = inputs.cuda(device=device)
        labels = labels.cuda(device=device)
        inputs = inputs.unsqueeze(1)  # 增加一個新的維度來表示通道
        outputs = model(inputs)

        _, predicted = torch.max(outputs.data, 1)

        for i in range(len(predicted)):
            pre_label.append((predicted[i].cpu()).item())
        true_label += (labels.cpu()).tolist()

        # totoal
        total += labels.size(0)
        total_correct += (predicted == labels).sum().item()
        total_false += (predicted != labels).sum().item()
        c = (predicted == labels).squeeze()
        for i in range(labels.size(0)):
            label = int(labels[i])
            class_correct[label] += c[i].item()
            class_total[label] += 1

    total_acc = 0
    with open(f"{subdir}/Test_Accuracy.txt", "w") as file:
        for i, c in enumerate(use_class):
            try:
                print(
                    "Accuracy of %5s : %8.2f %%"
                    % (c, 100 * class_correct[i] / class_total[i])
                )
                file.write(
                    "%5s: %8.2f %%\n"
                    % (c, 100 * class_correct[i] / class_total[i])
                )
                total_acc += (class_correct[i] /
                            class_total[i]) / len(use_class)
            except ZeroDivisionError:
                print("Accuracy of %5s : %8f %%" % (c, 0))
                file.write("%5s: %8.2f %%\n" % (c, 0))
                total_acc += 0
        print("\nAccuracy on the ALL test classes: %f %%" % (total_acc * 100))
        file.write("Accuracy on the ALL test classes: %f %%\n" % (total_acc * 100))

    f1 = f1_score(true_label, pre_label, average='macro', zero_division=1)
    print(f"f1: {f1}")
    

  0%|          | 0/855 [00:00<?, ?it/s]

Accuracy of benign :    97.68 %
Accuracy of    BF :   100.00 %
Accuracy of Deauth :   100.00 %
Accuracy of SAE_CV :   100.00 %
Accuracy of   UGD :   100.00 %
Accuracy of SAE_AF :    99.66 %
Accuracy of   TSC :   100.00 %
Accuracy of    DG :    96.77 %
Accuracy of KRACK_DG :    97.34 %

Accuracy on the ALL test classes: 99.049844 %
f1: 0.96091694802155


In [45]:
draw_confusion_matrix(true_label, pre_label, use_class, subdir)